## Notebook for computing annual average for sea ice extent

This notebook is intended to run annualized_timeseries.py in the notebook format for better readibility.  
For the most updated code to compute annual average for each region, please refer to annualized_timeseries.py.


In [1]:

from pw_data import SIC25k
import numpy as np
import pandas as pd
import geopandas as gpd
from dask.distributed import Client, LocalCluster

In [2]:

"""This code loads data and computes annualized time series 
sea ice extent and exports them in csv format
For data information, https://polarwatch.noaa.gov/erddap/info/nsidcG10016v2nhmday/index.html
monthly: 'nsidcG02202v4nhmday' (cdr_seaice_conc_monthly)
daily: nsidcG02202v4nh1day (cdr_seaice_conc)
"""


cluster = LocalCluster(n_workers=4, threads_per_worker=2)
client = Client(cluster)
print(f"Dashboard is running on: {client.dashboard_link}")

# monthly
erddap_id = 'nsidcG02202v4nh1day' 
area_id = 'pstere_gridcell_N25k'
crs = 'epsg:3413'
var_name = 'cdr_seaice_conc'

regions = dict([('AlaskanArctic', 'arctic_sf.shp'), ('NorthernBering', 'nbering_sf.shp'), 
            ('EasternBering', 'ebering_sf.shp')]) #, ('SouthernBering', 'sbering_sf.shp')])
# regions = dict([('AlaskanArctic', 'arctic_sf.shp')])

# Instantiate SIC25k instance and load sic and grid data            
sic_m = SIC25k(erddap_id, var_name, crs)
sic_m.load_area(area_id)


for name, shp in regions.items(): 
    # Load shape geometry and transform
    alaska_shp = gpd.read_file(f'resources/akmarineeco/{shp}')
    alaska_shp_proj = alaska_shp.to_crs(crs)  
    extents = []
    # Sep 1-Aug 31
    for year in range(1995, 2010):
        # clip to region and date
        ds, area = sic_m.subset_dim([f'{year}-09-01', f'{year+1}-08-31'], alaska_shp_proj)
        sic = sic_m.format_sic(ds, 0.15) # format data with threshold <= 0.15
        ext = sic_m.compute_extent_km(sic, area)
        extents.append({'region': name, 'year': year, 'extent': np.mean(ext.values)})
    df = pd.DataFrame(extents)
    df.to_csv(f'{name}_annual_extent_2010_today.csv', index=False)




Dashboard is running on: http://127.0.0.1:8787/status


/Users/sunbak/miniforge3/envs/seaice/lib/python3.9/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'melt_onset_day_cdr_seaice_conc' has multiple fill values {-1, 255} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


Unable to load data: 'NoneType' object is not subscriptable


/Users/sunbak/miniforge3/envs/seaice/lib/python3.9/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'melt_onset_day_cdr_seaice_conc' has multiple fill values {-1, 255} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
